In [1]:
import gcp.bigquery as bq
import pandas as pd
import numpy as np


%%bigquery schema --table "nyc-tlc:green.trips_2015"

In [2]:
%%sql
SELECT DAYOFYEAR(pickup_datetime) AS daynumber FROM [nyc-tlc:green.trips_2015] LIMIT 10

daynumber
120
120
120
149
149
150
150
89
89
89


In [3]:
%%sql --module taxiquery
SELECT daynumber, COUNT(*) AS numtrips FROM
    (SELECT DAYOFYEAR(pickup_datetime) AS daynumber FROM [nyc-tlc:green.trips_$YEAR])
GROUP BY daynumber ORDER BY daynumber


In [4]:
trips = bq.Query(taxiquery, YEAR=2015).to_dataframe()
trips[:5]

,daynumber,numtrips
0,1,62943
1,2,43410
2,3,53866
3,4,41602
4,5,41923
